<a href="https://colab.research.google.com/gist/priyanshusharma16/a3bc3042449f676aceba1265ac33f2df/untitled45.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ReduceLROnPlateau

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
# Load the data
data = pd.read_json('/content/Sarcasm_Headlines_Dataset.json', lines=True)

In [20]:
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


In [22]:
data.shape

(28619, 3)

In [23]:
#checking for null values in data
data.isnull().sum()

is_sarcastic    0
headline        0
article_link    0
dtype: int64

In [24]:
data.is_sarcastic.value_counts()

is_sarcastic
0    14985
1    13634
Name: count, dtype: int64

In [25]:
#checking for duplicate values
data['headline'].duplicated().sum()

116

In [26]:
# Drop duplicate headlines
data = data.drop(data[data['headline'].duplicated()].index, axis=0)

In [27]:
#rechecking for duplicate values
data['headline'].duplicated().sum()

0

In [28]:
# Drop unnecessary columns
data = data.drop(columns=['article_link'])

In [29]:
data.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


In [30]:
# Text preprocessing functions
stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(f'[{string.punctuation}]', '', text)
    # Remove stopwords and apply lemmatization and stemming
    words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in text.split() if word not in stop]
    return ' '.join(words)

data['clean_headline'] = data['headline'].apply(clean_text)

In [31]:
data.head()

,is_sarcastic,headline,clean_headline
0,1,thirtysomething scientists unveil doomsday clo...,thirtysometh scientist unveil doomsday clock h...
1,0,dem rep. totally nails why congress is falling...,dem rep total nail congress fall short gender ...
2,0,eat your veggies: 9 deliciously different recipes,eat veggi 9 delici differ recip
3,1,inclement weather prevents liar from getting t...,inclement weather prevent liar get work
4,1,mother comes pretty close to using word 'strea...,mother come pretti close use word stream corre...


In [32]:
# Tokenizing the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['clean_headline'])
sequences = tokenizer.texts_to_sequences(data['clean_headline'])
word_index = tokenizer.word_index

# Padding the sequences
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [33]:
# Splitting the data
labels = data['is_sarcastic'].values
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [34]:
# Building the model
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=128, input_length=max_length))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Defining callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)

# Training the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[reduce_lr])

Epoch 1/10
571/571 [==============================] - 192s 320ms/step - loss: 0.5167 - accuracy: 0.7374 - val_loss: 0.4210 - val_accuracy: 0.8044 - lr: 0.0010
Epoch 2/10
571/571 [==============================] - 178s 311ms/step - loss: 0.2764 - accuracy: 0.8909 - val_loss: 0.4348 - val_accuracy: 0.8103 - lr: 0.0010
Epoch 3/10
571/571 [==============================] - 173s 303ms/step - loss: 0.1455 - accuracy: 0.9475 - val_loss: 0.5732 - val_accuracy: 0.8003 - lr: 0.0010
Epoch 4/10
571/571 [==============================] - 176s 308ms/step - loss: 0.0828 - accuracy: 0.9725 - val_loss: 0.7642 - val_accuracy: 0.7867 - lr: 0.0010
Epoch 5/10
571/571 [==============================] - 174s 304ms/step - loss: 0.0533 - accuracy: 0.9824 - val_loss: 0.8675 - val_accuracy: 0.7873 - lr: 0.0010
Epoch 6/10
571/571 [==============================] - 174s 305ms/step - loss: 0.0347 - accuracy: 0.9891 - val_loss: 1.0314 - val_accuracy: 0.7849 - lr: 0.0010
Epoch 7/10
571/571 [==========================

In [35]:
# Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

179/179 [==============================] - 13s 73ms/step - loss: 1.3294 - accuracy: 0.7751
Test Accuracy: 0.7751
